# 인턴 교육 2주차 과제: Selenium을 이용한 유튜브 댓글 크롤링
- 과제 기한: 6월 21일까지
- 제출자: 배정환 인턴

**세부과제**

1. n_scroll 변수를 활용하여 스크롤 다운 5번
    - 스크롤 다운 함수를 만든다.

2. Pandas를 이용한 데이터프레임 저장
    - `columns: [UserID, 댓글텍스트, 좋아요 수]`
    - 모든 for문이 끝난 후 URL에서 video_id 추출해서 videoID column 추가
    - csv로 저장

3. 전체 함수 만들기
    - URL만 입력하면 바로 저장될 수 있도록 함수를 만든다.

**목표**
- 스크롤하면서 로딩되는 댓글들을 Selenium 크롤링을 이용해 수집하기
- 에러에 대한 처리를 하며 나중에 오랜만에 했을 때 생기는 오류에 대해서도 빠르고 쉽게 처리할 수 있도록 경험을 쌓는다.

**팁**
- [Selenium Documentation](https://www.selenium.dev/documentation/)
- [Selenium-Python Documentation](https://selenium-python.readthedocs.io/index.html)

### 크롬 창 생성 및 유튜브 주소로 접속하기

In [185]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [186]:
# 크롬 창 열기
options = Options()
driver = webdriver.Chrome(options = options)

# 창모드 전체화면으로 크기 늘리기
driver.maximize_window()

# 로딩시간
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 유튜브 주소 접속하기
url = "https://www.youtube.com/watch?v=0BWScn_OWPk"
driver.get(url)

# 유튜브 로딩시간
time.sleep(5)

In [178]:
# # 동영상 일시정지를 하여 로딩되는 속도 늘리기
# ActionChains(driver).send_keys('k').perform()

### 과제1. n_scroll 변수를 활용하여 스크롤 다운 5번
- 스크롤 다운 함수를 만든다.

In [187]:
# n_scroll variable 만들기
n_scroll = 5

In [189]:
# 스크롤 다운 함수 만들기

# 모든 스크롤 내리는 방법에 대한 문제점
#   - 댓글이 로딩이 되지 않는다.
#   - 스크롤을 위로 올려서 처음 댓글부터 떠야 그다음 댓글들이 나타난다.

# 해결책
#   - 스크롤을 맨 위로 올린다면 댓글들이 로딩이 되면서 한페이지에 스크롤 5번한 댓글들의 데이터를 잡을 수 있을거 같다.

def scroll_down(n_scroll):
    """함수

    Args:
        n_scroll (integer): 스크롤 하는 횟수
    """
    # 첫번째 방법
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    for i in range(n_scroll):
        # 끝까지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        # 스크롤 내린 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


    # 두번째 방법 -> page 전체가 내려가는 것이 아니라 스크롤바 크기만큼 내려간다.
    # for _ in range(n_scroll):
    #     # JavaScript를 사용하여 페이지 스크롤
    #     driver.execute_script("window.scrollBy(0, window.innerHeight);")
    #     # 각 스크롤 후 약간의 지연을 추가하여 페이지가 로드될 시간을 줍니다.
    #     time.sleep(1)
    #     wait = WebDriverWait(driver, 10)
    #     wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    # 세번째 방법
    # container=driver.find_element(By.TAG_NAME, "html")
    # before_top = driver.execute_script("return arguments[0].scrollTop", container)

    # for i in range(n_scroll):
    #     time.sleep(3)
    #     driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
    #     after_top = driver.execute_script("return arguments[0].scrollTop", container)
    #     before_top = after_top
        
    # 해결책1 - 페이지의 스크롤을 맨 위로 올리기 -> 로딩이 안됨
    # driver.execute_script("window.scrollTo(0,0);")
    # time.sleep(2)
    
    # 해결책2 - 페이지의 스크롤을 관련영상 5번째까지 올리기 -> 첫 스크롤까지만 데이터가 받아짐
    related_video = driver.find_elements(By.CSS_SELECTOR, '#dismissible')[5]
    driver.execute_script("arguments[0].scrollIntoView(true);", related_video)
    
    # 해결책3 - 스크롤 한번 더 내린다. 이번에 스크롤 길이만큼 내리며 
    for _ in range(10):
        time.sleep(5)
        # JavaScript를 사용하여 페이지 스크롤
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        
    
    # 댓글이 로드될 때까지 기다리기
    comments_section = WebDriverWait(driver, 10)
    comments_section.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#contents')))
scroll_down(n_scroll)

**결론**
- 결국엔 스크롤을 15번을 돌리는 것이라 비효율적이다.
- 5번 스크롤 내리고 마지막 10번은 처음 5번과 비슷한 크기의 데이터양을 가지고 올 수 있다.
    - 비슷한 양의 데이터를 가지고 오는 것이 아니라 정확한 다섯 번 스크롤한 양의 데이터를 가지고 와야하지만
    - 결과는 다섯 번 스크롤한 양과 비슷한 양의 데이터만 가지고 온 것이다.
- 페이지 스크롤에 대한것은 웹서칭을 통해 다른 개발자들의 코드를 확인해보며 문제를 확인한다.

### 과제2.Pandas를 이용한 데이터프레임 저장
- columns: `[UserID, 댓글 텍스트, 좋아요 수]`

In [132]:
# UserID 가져오기
user_id = driver.find_elements(By.CSS_SELECTOR, '#author-text')


# 댓글 텍스트 가져오기
comment_text = driver.find_elements(By.CSS_SELECTOR, '#content-text')


# 좋아요 수 가져오기
num_likes = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')


@kimsoonsoo6004
@user-nt9zc3bk5d
@user-hw6tt8ke1n
@user-vv8bw2sx5n
@user-jt4qk9sx5v
@user-ff9po2nn6s
@zimbra67
@user-th3mk7kc4y
@user-ke2cz6nn7k
@boramskyhan


- for문을 통해 데이터 받기

In [190]:
# 데이터를 담을 빈 딕셔너리 만들기
comment_data_dict = {"UserID": [], "댓글텍스트":[],"좋아요 수":[]}

# 댓글 갯수를 나타내는 변수 지정하기
number_comments = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-comment-view-model')

# loop를 통해 데이터 받기 및 저장하기
for i in range(len(number_comments)):
    user_id = driver.find_elements(By.CSS_SELECTOR, '#author-text')[i].text
    comment_text = driver.find_elements(By.CSS_SELECTOR, "#content-text")[i].text
    num_likes = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')[i].text
    
# comment_data_dict 딕셔너리에 데이터 넣기
    comment_data_dict["UserID"].append(user_id)
    comment_data_dict["댓글텍스트"].append(comment_text)
    comment_data_dict["좋아요 수"].append(num_likes)


- 모든 For문이 끝난 후 URL에서 video_id 추출

In [191]:
# URL에서 video_id 추출하기
video_id = url.split('?v=')[1]


- 데이터프레임 만들기
- videoID column에 추가

In [192]:
# 데이터프레임으로 만들기
df_comments = pd.DataFrame(comment_data_dict)

# videoID column 추가
df_comments["video_ID"] = video_id

df_comments

,UserID,댓글텍스트,좋아요 수,video_ID
0,@kimsoonsoo6004,김하성의 영상엔 김하성이 한번도 등장하지 않는다.,328,0BWScn_OWPk
1,@user-nt9zc3bk5d,김하성의 야구엔 무안타의 감동이있다,183,0BWScn_OWPk
2,@user-hw6tt8ke1n,썸넬 김하성이 끝내기홈런친줄..ㅡㅡ,98,0BWScn_OWPk
3,@user-vv8bw2sx5n,샌디에이고의 2경기 연속 끝내기 홈런에는 언제나 감동 감탄 환호 멋진 플레이가있다,10,0BWScn_OWPk
4,@user-jt4qk9sx5v,썸네일 제목 뽑은 거 보니 김하성 무안타 였나보군?,38,0BWScn_OWPk
...,...,...,...,...
95,@idx8722,영상 시작하자마지 댓글보고 영상 바로 닫는다.,,0BWScn_OWPk
96,@mandol_ee,"FA를 까먹은 김하성,,",,0BWScn_OWPk
97,@user-kt5xu1sr3y,김하성이 친줄알았다,,0BWScn_OWPk
98,@user-eg7fd1jr8k,샌디에이고의 김하성 아님?ㅋㅋㅋㅋㅋ\n김하성의 샌디에이고???,,0BWScn_OWPk


In [184]:
# .csv파일로 저장
df_comments.to_csv("./youtube_comments_crawling.csv")

### 과제3. 전체 함수 만들기
- URL만 입력하면 바로 저장될 수 있도록 함수를 만든다.

In [200]:
# 코드블럭의 내용들을 한곳으로 모은다.
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def get_youtube_comment(url:str):
    """유튜브 댓글들, 댓글들의 아이디정보, 좋아요수를 추출하는 함수이다.

    Args:
        url (str): 추출할 유튜브 동영상 주소이다.
    """
    # 크롬 창 열기
    options = Options()
    driver = webdriver.Chrome(options = options)

    # 창모드 전체화면으로 크기 늘리기
    driver.maximize_window()
    
    # 로딩시간
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    # 유튜브 주소 접속하기
    # url = "https://www.youtube.com/watch?v=0BWScn_OWPk"
    driver.get(url)

    # videoID 찾기
    video_id = url.split('?v=')[1]
    
    # 유튜브 로딩시간
    time.sleep(5)
    
    # n_scroll 변수 만들기
    n_scroll = 5
    
    # scroll_down 함수 호출해서 스크롤 내리기
    scroll_down(driver,n_scroll)
    
    # 데이터를 담을 빈 딕셔너리 만들기
    comment_data_dict = {"UserID": [], "댓글텍스트":[],"좋아요 수":[]}

    # 댓글 갯수를 나타내는 변수 지정하기
    number_comments = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-comment-view-model')

    # loop를 통해 데이터 받기 및 저장하기
    for i in range(len(number_comments)):
        user_id = driver.find_elements(By.CSS_SELECTOR, '#author-text')[i].text
        comment_text = driver.find_elements(By.CSS_SELECTOR, "#content-text")[i].text
        num_likes = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')[i].text
    
    # comment_data_dict 딕셔너리에 데이터 넣기
        comment_data_dict["UserID"].append(user_id)
        comment_data_dict["댓글텍스트"].append(comment_text)
        comment_data_dict["좋아요 수"].append(num_likes)
        
    # DataFrame으로 만들기
    get_dataframe_and_csv(comment_data_dict,video_id)
    
    # 창 닫기
    driver.quit()
    
    
        
# =======================================================================================================================================
    
def scroll_down(driver, n_scroll:int):
    """정해진 횟수의 스크롤을 내리는 함수

    Args:
        driver (): 현재 창을 말한다.
        n_scroll (int): 스크롤 횟수이다.

    Returns:
        None : 
    """
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    for i in range(n_scroll):
        # 끝까지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        # 스크롤 내린 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # 페이지의 스크롤을 관련영상 5번째까지 올리기 -> 첫 스크롤까지만 데이터가 받아짐
    related_video = driver.find_elements(By.CSS_SELECTOR, '#dismissible')[5]
    driver.execute_script("arguments[0].scrollIntoView(true);", related_video)
    
    # 스크롤 한번 더 내린다. 이번에 스크롤 길이만큼 내리며 
    for _ in range(10):
        time.sleep(5)
        # JavaScript를 사용하여 페이지 스크롤
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        
    
    # 댓글이 로드될 때까지 기다리기
    comments_section = WebDriverWait(driver, 10)
    comments_section.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#contents')))

# =============================================================================================================================
def get_dataframe_and_csv(comment_data_dict: dict, video_id:str):
    """받은 데이터를 데이터프레임으로 만들어 csv파일에 저장하는 함수

    Args:
        comment_data_dict (dict): 유튜브 댓글, 아이디 정보, 좋아요 수를 추출하여 저장 공간이다.
        video_id (str): 유튜브 동영상 주소 뒤에 있는 고유 주소이다.
    """
    # 데이터프레임으로 만들기
    df_comments = pd.DataFrame(comment_data_dict)

    # videoID column 추가
    df_comments["video_ID"] = video_id
    
    # .csv파일로 저장
    df_comments.to_csv("./youtube_comments_crawling.csv")

# =====================================================================================================
# 입력칸
url = "https://www.youtube.com/watch?v=0BWScn_OWPk"
get_youtube_comment(url)